<a href="https://colab.research.google.com/github/amalsalilan/IPL_Infographics_Data_Analytics_-_Data_Visualization_Infosys_Internship_Oct2024/blob/D_Aasritha_Reddy/UI_Prediction_IPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00


In [19]:
%%writefile app1.py
import pandas as pd
import numpy as np
import streamlit as st
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Team mappings
team_mapping = {
    'GT': 0, 'PBKS': 1, 'LSG': 2, 'SRH': 3, 'RCB': 4, 'CSK': 5, 'DC': 6, 'RR': 7,
    'KKR': 8, 'MI': 9, 'KXIP': 10, 'RPS': 11, 'GL': 12, 'PWI': 13, 'KOCHI': 14
}
reverse_team_mapping = {v: k for k, v in team_mapping.items()}

# Function to load model and data
@st.cache_data
def load_data():
    model_df = pd.read_csv('modelling_final.csv')
    return model_df

st.set_page_config(page_title="Cumulative Runs Prediction", layout="wide")
st.title("Cumulative Runs Prediction Using Random Forest Regressor")

# Sidebar - File upload for custom data
st.sidebar.header("Upload Your Data")
uploaded_file = st.sidebar.file_uploader("Choose a CSV file", type=["csv"])

if uploaded_file is not None:
    model_df = pd.read_csv(uploaded_file)
else:
    model_df = load_data()

st.write("Data Overview:")
st.write(model_df.head())

# Define features and target variable
target = 'cumulative_runs'
features = ['over', 'ball', 'innings_id','current_innings', 'home_teams', 'away_teams', 'run_rate', 'wickets_lost']

# Map team names in dataset if needed
if 'innings_id' in model_df.columns and model_df['innings_id'].dtype == object:
    model_df['innings_id'] = model_df['innings_id'].map(team_mapping)
if 'home_teams' in model_df.columns and model_df['home_teams'].dtype == object:
    model_df['home_teams'] = model_df['home_teams'].map(team_mapping)
if 'away_teams' in model_df.columns and model_df['away_teams'].dtype == object:
    model_df['away_teams'] = model_df['away_teams'].map(team_mapping)

X = model_df[features]
y = model_df[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train RandomForest model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Streamlit form to take input for prediction
st.sidebar.header("Enter match details for prediction:")
over = st.sidebar.number_input("Over", min_value=0, max_value=50, value=0)
ball = st.sidebar.number_input("Ball", min_value=0, max_value=6, value=0)
innings_id = st.sidebar.number_input("Innings", min_value=1, max_value=2, value=1)
current_innings = st.sidebar.selectbox("Current Innings", options=list(team_mapping.keys()))
home_team = st.sidebar.selectbox("Home Team", options=list(team_mapping.keys()))
away_team = st.sidebar.selectbox("Away Team", options=list(team_mapping.keys()))
run_rate = st.sidebar.number_input("Run Rate", min_value=0.0, max_value=20.0, value=0.0)
wickets_lost = st.sidebar.number_input("Wickets Lost", min_value=0, max_value=10, value=0)

if st.sidebar.button("Predict Cumulative Runs"):
    # Encode team names using team_mapping
    home_team_encoded = team_mapping[home_team]
    away_team_encoded = team_mapping[away_team]
    input_data = np.array([
        over, ball, innings_id,current_innings, home_team_encoded, away_team_encoded, run_rate, wickets_lost
    ]).reshape(1, -1)
    predicted_run = model.predict(input_data)
    st.write(f"Predicted Cumulative Runs: {predicted_run[0]}")

if st.button("Evaluate Model"):
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    st.write(f"Mean Squared Error: {mse:.3f}")
    st.write(f"Root Mean Squared Error: {rmse:.3f}")
    st.write(f"R-squared: {r2:.3f}")


Overwriting app1.py


In [20]:
from pyngrok import ngrok

# Replace with your ngrok authtoken
ngrok.set_auth_token('2pQHN3PjzsDU7GcuyPZQOAsXBIh_3zVzewmj64hcwCQjRLhtf')

# Create a tunnel to expose the Streamlit app
public_url = ngrok.connect(8501)
print('Streamlit app is live at:', public_url)

# Run Streamlit app
!streamlit run app1.py &


Streamlit app is live at: NgrokTunnel: "https://2074-35-199-29-32.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.199.29.32:8501



  Stopping...
